In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [3]:
def loadData():
    with np.load("notMNIST.npz") as data:
        Data, Target = data["images"], data["labels"]
        np.random.seed(521)
        randIndx = np.arange(len(Data))
        np.random.shuffle(randIndx)
        Data = Data[randIndx] / 255.0
        Target = Target[randIndx]
        trainData, trainTarget = Data[:10000], Target[:10000]
        validData, validTarget = Data[10000:16000], Target[10000:16000]
        testData, testTarget = Data[16000:], Target[16000:]
    return trainData, validData, testData, trainTarget, validTarget, testTarget


In [4]:
def convertOneHot(trainTarget, validTarget, testTarget):
    newtrain = np.zeros((trainTarget.shape[0], 10))
    newvalid = np.zeros((validTarget.shape[0], 10))
    newtest = np.zeros((testTarget.shape[0], 10))

    for item in range(0, trainTarget.shape[0]):
        newtrain[item][trainTarget[item]] = 1
    for item in range(0, validTarget.shape[0]):
        newvalid[item][validTarget[item]] = 1
    for item in range(0, testTarget.shape[0]):
        newtest[item][testTarget[item]] = 1
    return newtrain, newvalid, newtest


In [5]:
def shuffle(trainData, trainTarget):
    np.random.seed(421)
    randIndx = np.arange(len(trainData))
    target = trainTarget
    np.random.shuffle(randIndx)
    data, target = trainData[randIndx], target[randIndx]
    return data, target



In [6]:
trainData, validData, testData, trainTarget, validTarget, testTarget= loadData()
train_y, valid_y, test_y = convertOneHot( trainTarget, validTarget, testTarget)
#Train Data 
train_x = trainData

#X0 = np.ones((3500,1))
#train_X = np.append(X0,train_X,axis=1)

#Test Data
test_X = testData


#Validation Data
validation_X = validData

In [7]:
training_epoch = 50 
learning_rate = 0.0001 
mini_batch_size = 32
resuf = 10000 / mini_batch_size
classes_out = 10
total_iterations =int(training_epoch * 10000 / mini_batch_size)


In [8]:
x = tf.placeholder("float", [None, 28,28,1])
y = tf.placeholder("float", [None, classes_out])

In [9]:

W1 = tf.get_variable("W", shape=[5,5,1,32],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32)
b1 = tf.get_variable("b", shape=[32],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32)

W2 = tf.get_variable("Wf2", shape=[14*14*32,32],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32)
b2 = tf.get_variable("bf2", shape=[32],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32)

W3 = tf.get_variable("Wf3", shape=[32,classes_out],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32)
b3 = tf.get_variable("bf3", shape=[classes_out],initializer=tf.contrib.layers.xavier_initializer(),dtype=tf.float32)

In [11]:
conv1 = tf.nn.conv2d(x, W1, strides=[1, 1, 1, 1], padding='SAME')
conv1 = tf.nn.bias_add(conv1, b1)
conv1 = tf.nn.relu(conv1)

#####NEEDED TO BE CHECKED########
batch_mean, batch_var = tf.nn.moments(conv1,[0, 1, 2])
Batch_normalization = tf.nn.batch_normalization(conv1,batch_mean,batch_var,None,None,1e-5)

#check 2 x 2 
max_pool = tf.nn.max_pool(Batch_normalization, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                          padding='SAME')

fc1 = tf.reshape(max_pool, [-1, W2.get_shape()[0]])
fc1 = tf.add(tf.matmul(fc1, W2), b2)
fc1 = tf.nn.relu(fc1)

fc2 = tf.reshape(fc1, [-1, W3.get_shape()[0]])
fc2 = tf.add(tf.matmul(fc2, W3), b3)

pred = tf.nn.softmax(fc2)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()

In [ ]:
train_x = train_x.reshape(-1, 28, 28, 1)
#train_x = tf.cast(train_x, tf.float32)
validation_x = validation_X.reshape(-1, 28, 28, 1)
test_x = test_X.reshape(-1, 28, 28, 1)

with tf.Session() as sess:
    sess.run(init) 
    train_loss = []
    test_loss = []
    validation_loss = []
    train_accuracy = []
    validation_accuracy = []
    test_accuracy= []
    total_iter=[]
    
    for iter in range(total_iterations):
        total_iter.append(iter)
        image_up = train_x[(iter*mini_batch_size)%10000:((iter*mini_batch_size)%10000)+mini_batch_size]
        target_up = train_y[(iter*mini_batch_size)%10000:((iter*mini_batch_size)%10000)+mini_batch_size]
        opt = sess.run(optimizer, feed_dict={x: image_up,y: target_up})
        
        loss, acc = sess.run([cost, accuracy], feed_dict={x: image_up,y: target_up})
        train_loss.append(loss)
        train_accuracy.append(acc)
        
        
        valid_cost,valid_accu = sess.run([cost, accuracy], feed_dict={x: validation_x,y : valid_y})
        validation_loss.append(valid_cost)
        validation_accuracy.append(valid_accu)
        
        
        loss_test, acc_test = sess.run([cost, accuracy], feed_dict={x: test_x,y: test_y})
        train_loss.append(loss_test)
        train_accuracy.append(acc_test)
        
        print("Iter " + str(iter) + ", Train Loss:" + "{:.3f}".format(loss) + ", Train Acc:" + "{:.3f}".format(acc) + ", Valid Loss:" + "{:.3f}".format(valid_cost) + ", Valid Acc:" + "{:.3f}".format(valid_accu) + ", Test Loss:" + "{:.3f}".format(loss_test) + ", Test Acc:" + "{:.3f}".format(acc_test))
        if iter% int(resuf) == 0 :
            train_x ,  train_y = shuffle(trainData=train_x,trainTarget=train_y)

Iter 0, Train Loss:2.315, Train Acc:0.062, Valid Loss:2.313, Valid Acc:0.084, Test Loss:2.314, Test Acc:0.079
Iter 1, Train Loss:2.322, Train Acc:0.062, Valid Loss:2.304, Valid Acc:0.097, Test Loss:2.305, Test Acc:0.093
Iter 2, Train Loss:2.295, Train Acc:0.094, Valid Loss:2.292, Valid Acc:0.121, Test Loss:2.293, Test Acc:0.116
Iter 3, Train Loss:2.272, Train Acc:0.125, Valid Loss:2.278, Valid Acc:0.167, Test Loss:2.280, Test Acc:0.158
Iter 4, Train Loss:2.305, Train Acc:0.094, Valid Loss:2.264, Valid Acc:0.194, Test Loss:2.267, Test Acc:0.186
Iter 5, Train Loss:2.223, Train Acc:0.281, Valid Loss:2.250, Valid Acc:0.226, Test Loss:2.253, Test Acc:0.215
Iter 6, Train Loss:2.236, Train Acc:0.250, Valid Loss:2.236, Valid Acc:0.249, Test Loss:2.239, Test Acc:0.240
Iter 7, Train Loss:2.185, Train Acc:0.375, Valid Loss:2.221, Valid Acc:0.265, Test Loss:2.225, Test Acc:0.259
Iter 8, Train Loss:2.200, Train Acc:0.312, Valid Loss:2.207, Valid Acc:0.285, Test Loss:2.212, Test Acc:0.280
Iter 9, Tr